In [8]:
import cv2
import mediapipe as mp
import math
import numpy as np

# -----------------------------
# MediaPipe Initialization
# -----------------------------
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

finger_joints = {
    "thumb": [1, 2, 3, 4],
    "index": [5, 6, 7, 8],
    "middle": [9, 10, 11, 12],
    "ring": [13, 14, 15, 16],
    "pinky": [17, 18, 19, 20]
}

# -----------------------------
# Ring Size Table (US)
# -----------------------------
def diameter_to_us_ring_size(mm):
    table = {
        14.0: 3, 14.4: 3.5, 14.8: 4, 15.2: 4.5, 15.6: 5,
        16.0: 5.5, 16.5: 6, 16.9: 6.5, 17.3: 7, 17.7: 7.5,
        18.1: 8, 18.5: 8.5, 19.0: 9, 19.4: 9.5, 19.8: 10,
        20.2: 10.5, 20.6: 11, 21.0: 11.5, 21.4: 12, 21.8: 12.5, 22.2: 13
    }
    closest = min(table.keys(), key=lambda x: abs(x - mm))
    return table[closest]

# -----------------------------
# Distance Calculation
# -----------------------------
def euclidean_distance(p1, p2, w, h):
    x1, y1 = int(p1.x * w), int(p1.y * h)
    x2, y2 = int(p2.x * w), int(p2.y * h)
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2), (x1, y1), (x2, y2)

def measure_fingers(image, results, scale=1.0):
    h, w, _ = image.shape
    measurements = {}
    for hand_landmarks in results.multi_hand_landmarks:
        mp_drawing.draw_landmarks(
            image, 
            hand_landmarks, 
            mp_hands.HAND_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=1, circle_radius=1),  # thinner landmarks
            mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=1, circle_radius=1)   # thinner connections
        )

        for finger, ids in finger_joints.items():
            left_point = hand_landmarks.landmark[ids[1]]
            right_point = hand_landmarks.landmark[ids[2]]

            dist_pixels, p1, p2 = euclidean_distance(left_point, right_point, w, h)
            dist_mm = dist_pixels * scale  # Convert pixels to mm if scale provided
            measurements[finger] = dist_mm

            cv2.line(image, p1, p2, (0, 255, 0), 1)  # thinner line
            cv2.putText(image, f"{finger}: {dist_mm:.1f}mm", (p1[0], p1[1]-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0,0,255), 1)  # thinner text
            cv2.putText(image, f"US Ring: {diameter_to_us_ring_size(dist_mm)}", (p1[0], p1[1]-25),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255,0,0), 1)  # thinner text
    return image, measurements

# -----------------------------
# Photo Mode
# -----------------------------
def photo_mode(image_path, reference_mm=None, reference_pixels=None):
    image = cv2.imread(image_path)
    scale = 1.0
    if reference_mm and reference_pixels:
        scale = reference_mm / reference_pixels  # mm per pixel

    with mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.7) as hands:
        rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = hands.process(rgb)
        if not results.multi_hand_landmarks:
            print("No hand detected")
            return
        image, meas = measure_fingers(image, results, scale)
        print("Finger measurements (mm):", meas)
        cv2.imshow("Hand Measurement", image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

# -----------------------------
# Video Mode
# -----------------------------
def video_mode(reference_mm=None, reference_pixels=None):
    scale = 1.0
    if reference_mm and reference_pixels:
        scale = reference_mm / reference_pixels  # mm per pixel

    cap = cv2.VideoCapture(0)
    with mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.7, min_tracking_confidence=0.7) as hands:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = hands.process(rgb)
            if results.multi_hand_landmarks:
                frame, meas = measure_fingers(frame, results, scale)
            cv2.imshow("Finger Measurement (Press Q to quit)", frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    cap.release()
    cv2.destroyAllWindows()

# -----------------------------
# Example Usage
# -----------------------------
# 1) Photo mode with optional reference object
# photo_mode("hand.jpg", reference_mm=24, reference_pixels=50)  # e.g., coin 24mm spans 50 pixels in image

# 2) Webcam / Video mode
video_mode(reference_mm=None, reference_pixels=None)  # Use None for pixel measurements only
